rules of the game : https://cdn.1j1ju.com/medias/85/5e/7c-pickomino-rulebook.pdf

subject : https://team.inria.fr/polaris/files/2023/10/project_2023.pdf

#PART 1: MDP for 1 player

\go faire un truc déguelasse avec dico et sets et ensuite on optimise si besoin

on considère dans toute la suite que 6 est synonyme de 'W' (worm)

a state is : a sorted list of dices rolled + a set of faces you cannot choose + the sum of dices already picked

In [1]:
#Première question
from dice_state import *
from mdp import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.opti[state], "value is:", mdp.value[state])


Continue with dice 4 value is: 1.232827668101253


In [2]:
#For c = -3
c = -3 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

state = DiceState(dices, score, used)
mdp.explore(state)


print(mdp.opti[state], "value is:", mdp.value[state])


Continue with dice 4 value is: 0.2562805612151674


In [4]:
#Probability of obtaining a tile with 1 or more worms
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.6866008010434065


In [6]:
#Probability of obtaining a tile with 2 or more worms
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*4+[1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.4248021494233747


In [7]:
#Probability of obtaining a tile with 3 or more worms
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*8+[1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.21179779018916067


In [8]:
#Probability of obtaining a tile with 4 worms
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*12+[1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.07352474185203971


In [2]:
#Probability of getting exactly tile 24
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,1] + [0]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print("Probability of getting exactly tile 24 is",p)


Probability of getting exactly tile 24 is 0.24243906674019267


In [3]:
#Probability of getting tile 27 or more
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,0,0,0,1] + [1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print("Probability of getting tile 27 of more is",p)

Probability of getting tile 27 of more is 0.310248006941


In [1]:
#Verification mdp
from dice_state import *
from mdp import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 1, 2, 4, 4, 4, 5, 6)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.value[state], mdp.opti[state])

1.0262671744380285 Continue with dice 4


In [2]:
#Verification mdp
from dice_state import *
from mdp import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 4, 4, 6)
score = 9
used = {3}
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.value[state], mdp.opti[state])
# state2 = (new_score, tuple(new_used), nb_dices-count)
#keeping the 2 four
state2 = (9+8,tuple({3,4}),3)
print("The expected value for keeping the 2 four is:",mdp.mem2[state2])

0.6304012345679011 Continue with dice 4
The expected value for keeping the 2 four is: 0.6304012345679011


In [7]:
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(0, 2)
n1 = f"AlphaBeta(0, 2)"
p2 = AlphaBetaPlayer(1,1)
n2 = f"AlphaBeta(1, 1)"

NUM_GAMES = 500

w, t = 0, 0
for i in range(NUM_GAMES):
    if i % 2 == 0:
        winner = run_game(p1, p2)
    else:
        winner = run_game(p2, p1)
    if winner == PlayerTurn.PLAYER_1:
        w+=1
    t += 1
    print(f"[{i}] {n1} wins with probabilty {w/t} over {n2}")



[0] AlphaBeta(0, 2) wins with probabilty 0.0 over AlphaBeta(1, 1)
[1] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta(1, 1)
[2] AlphaBeta(0, 2) wins with probabilty 0.3333333333333333 over AlphaBeta(1, 1)
[3] AlphaBeta(0, 2) wins with probabilty 0.25 over AlphaBeta(1, 1)
[4] AlphaBeta(0, 2) wins with probabilty 0.4 over AlphaBeta(1, 1)
[5] AlphaBeta(0, 2) wins with probabilty 0.3333333333333333 over AlphaBeta(1, 1)
[6] AlphaBeta(0, 2) wins with probabilty 0.42857142857142855 over AlphaBeta(1, 1)
[7] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta(1, 1)
[8] AlphaBeta(0, 2) wins with probabilty 0.5555555555555556 over AlphaBeta(1, 1)
[9] AlphaBeta(0, 2) wins with probabilty 0.6 over AlphaBeta(1, 1)
[10] AlphaBeta(0, 2) wins with probabilty 0.6363636363636364 over AlphaBeta(1, 1)
[11] AlphaBeta(0, 2) wins with probabilty 0.6666666666666666 over AlphaBeta(1, 1)
[12] AlphaBeta(0, 2) wins with probabilty 0.6923076923076923 over AlphaBeta(1, 1)
[13] AlphaBeta(0, 2) wins with p

In [8]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *

p1 = AlphaBetaPlayer(1,1)
p2 = AlphaBetaPlayer(1.5,0.5)

w = run_game(p1,p2)
print(w)

PlayerTurn.PLAYER_2


In [9]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from mdp import *

m = MDP(0, [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4])

state = DiceState((2, 4, 6, 6, 6), 15, {4})

m.explore(state)

print(m.opti[state])
for i in m.opti:
    print(i,m.opti[i])

Continue with dice 6
() 21 {1, 2, 4} Lose
(1, 1, 1, 1) 17 {2, 4} Continue with dice 1
(1,) 20 {1, 2, 4} Lose
(2,) 20 {1, 2, 4} Lose
() 23 {1, 2, 3, 4} Lose
(3,) 20 {1, 2, 4} Continue with dice 3
(4,) 20 {1, 2, 4} Lose
() 25 {1, 2, 4, 5} Lose
(5,) 20 {1, 2, 4} Continue with dice 5
() 25 {1, 2, 4, 6} Stop and pick tile 4
(6,) 20 {1, 2, 4} Continue with dice 6
(1, 1, 1, 2) 17 {2, 4} Continue with dice 1
(1, 1, 1) 20 {2, 3, 4} Continue with dice 1
(1,) 22 {1, 2, 3, 4} Lose
(2,) 22 {1, 2, 3, 4} Lose
(3,) 22 {1, 2, 3, 4} Lose
(4,) 22 {1, 2, 3, 4} Lose
() 27 {1, 2, 3, 4, 5} Lose
(5,) 22 {1, 2, 3, 4} Continue with dice 5
() 27 {1, 2, 3, 4, 6} Stop and pick tile 6
(6,) 22 {1, 2, 3, 4} Continue with dice 6
(1, 1, 2) 20 {2, 3, 4} Continue with dice 1
(1, 1, 3) 20 {2, 3, 4} Continue with dice 1
(1, 1, 4) 20 {2, 3, 4} Continue with dice 1
(1, 1) 25 {2, 3, 4, 5} Continue with dice 1
(1,) 26 {1, 2, 3, 4, 5} Lose
(2,) 26 {1, 2, 3, 4, 5} Lose
(3,) 26 {1, 2, 3, 4, 5} Lose
(4,) 26 {1, 2, 3, 4, 5} Lose
(5

In [1]:
from ucb import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(1,1)

num_games = 10000
grid_size = 9

best_pair = simulate_games_ucb(p1, grid_size, num_games)
print(f"The best pair against (1,1) is: {best_pair}")

[1]Pair (0.0, 0.0) won. Winrate: of 1.00 (1 / 1)
[2]Pair (0.0, 0.25) won. Winrate: of 1.00 (1 / 1)
[3]Pair (0.0, 0.5) won. Winrate: of 1.00 (1 / 1)
[4]Pair (0.0, 0.75) lost. Winrate: of 0.00 (0 / 1)
[5]Pair (0.0, 1.0) won. Winrate: of 1.00 (1 / 1)
[6]Pair (0.0, 1.25) won. Winrate: of 1.00 (1 / 1)
[7]Pair (0.0, 1.5) lost. Winrate: of 0.00 (0 / 1)
[8]Pair (0.0, 1.75) won. Winrate: of 1.00 (1 / 1)
[9]Pair (0.0, 2.0) won. Winrate: of 1.00 (1 / 1)
[10]Pair (0.25, 0.0) lost. Winrate: of 0.00 (0 / 1)
[11]Pair (0.25, 0.25) lost. Winrate: of 0.00 (0 / 1)
[12]Pair (0.25, 0.5) won. Winrate: of 1.00 (1 / 1)
[13]Pair (0.25, 0.75) lost. Winrate: of 0.00 (0 / 1)
[14]Pair (0.25, 1.0) won. Winrate: of 1.00 (1 / 1)
[15]Pair (0.25, 1.25) lost. Winrate: of 0.00 (0 / 1)
[16]Pair (0.25, 1.5) lost. Winrate: of 0.00 (0 / 1)
[17]Pair (0.25, 1.75) won. Winrate: of 1.00 (1 / 1)
[18]Pair (0.25, 2.0) lost. Winrate: of 0.00 (0 / 1)
[19]Pair (0.5, 0.0) lost. Winrate: of 0.00 (0 / 1)
[20]Pair (0.5, 0.25) lost. Winrat

In [13]:
from reinforce import *
from constants import *
from feature_map import *


#doit être de la même taile que phi
w0 = np.array([1 for i in range(10)])

pr = Reinforce(w0,1,1,phi,0.95)


In [26]:
pr.train(2000,0.01)

0
[ 5.43225241 -1.43174762  0.51232991  3.28863798  1.77386441  5.74929654
 -0.0926817   1.78243211  1.12178889  1.1251482 ]
gradient norm: 0.4824743834220151
100
[ 5.40311085 -1.42303096  0.51111473  3.31797935  1.82152035  5.80840076
 -0.14528355  1.80386091  1.12129877  1.12669721]
gradient norm: 0.8607232114946788
200
[ 5.43012684 -1.43659554  0.51492313  3.26910963  1.85013442  5.8630789
 -0.16590502  1.78849274  1.12129847  1.12380288]
gradient norm: 0.838604552451886
300
[ 5.51281104 -1.41363484  0.51810243  3.26786861  1.86416445  5.92591646
 -0.1808246   1.80009294  1.11795201  1.12796133]
gradient norm: 0.0010396741002088761
400
[ 5.54385713 -1.47223738  0.52015987  3.22293666  1.88669441  6.00474444
 -0.19555928  1.78804435  1.11501861  1.12406422]
gradient norm: 0.0049957402708449665
500
[ 5.49744219 -1.51577265  0.50311741  3.3225689   1.92565617  6.07819063
 -0.24953737  1.82666302  1.1174883   1.13157239]
gradient norm: 9.604809731298543e-05
600
[ 5.47589994 -1.45012239 

In [27]:
pr.evaluate(1000)

0.603

In [12]:
print(pr.w)

[1 1 1 1 1 1 1 1 1 1]
